In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 22.0 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import os

# Upload the zip file
uploaded = files.upload()

Saving JN-1-A.pdf to JN-1-A (1).pdf
Saving JN-1-D.pdf to JN-1-D (1).pdf
Saving JN-8-A.pdf to JN-8-A (1).pdf
Saving JN-8-D.pdf to JN-8-D (1).pdf
Saving JN-10-A.pdf to JN-10-A (1).pdf
Saving JN-10-D.pdf to JN-10-D (1).pdf
Saving JN-11-A.pdf to JN-11-A (1).pdf
Saving JN-11-D.pdf to JN-11-D (1).pdf
Saving JN-12-A.pdf to JN-12-A (1).pdf
Saving JN-12-D.pdf to JN-12-D (1).pdf
Saving JN-13-A.pdf to JN-13-A (1).pdf
Saving JN-13-D.pdf to JN-13-D (1).pdf
Saving JN-701-A.pdf to JN-701-A (1).pdf
Saving JN-701-D.pdf to JN-701-D (1).pdf
Saving JN-702-A.pdf to JN-702-A (1).pdf
Saving JN-702-D.pdf to JN-702-D (1).pdf


In [ ]:
from collections import namedtuple
import pandas as pd
import pdfplumber
import re
Output = pd.DataFrame(columns= ['Scenario','Node','V/C', 'Delay', 'LOS'] )

In [ ]:
for filename in uploaded.keys():
    all_text = []
    with pdfplumber.open(filename) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:  # Ensure the page contains text
                all_text.append(f"Page {page_num}:\n{page_text}\n")

    # Join the text of all pages into a single string, separating by newlines
    full_text = "\n".join(all_text)
    data = []
    for line in full_text.split('\n'):
        data.append(line)
    match = re.match(r"(\S+ \d+)\s(.+)", data[2])
    if match:
        node = match.group(1)  # Extract "Jn X"
        Scenario = match.group(2)
    data_string = "\n".join(data)
    data_string = re.sub(r"\nMovement", r"\n Movement", data_string)
    data_string = re.sub(r"\nIntersection", r"\n Intersection", data_string)
    pattern = r"Movement (.*?)Critical Lane Group"
    match = re.search(pattern, data_string, re.DOTALL)

    if match:
        Movemt = match.group(1).strip()  # Extracted content
    pattern = r"Intersection Summary(.*)"
    match = re.search(pattern, Movemt, re.DOTALL)

    if match:
        extracted_data = match.group(1).strip()

    pattern = r"HCM 2000 Control Delay (\S*) .*?Level of Service (\S*).*?Volume to Capacity ratio (\S*)"

    match = re.search(pattern, extracted_data, re.DOTALL)

    if match:
        control_delay, level_of_service, v_c_ratio = match.groups()
    new_row = {
        'Scenario': Scenario,
        'Node': node,
        'V/C':v_c_ratio,
        'Delay': control_delay,
        'LOS': level_of_service
    }

    # Use pd.concat to add the new row
    Output = pd.concat([Output, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
Output

,Scenario,Node,V/C,Delay,LOS
0,Two at grade - GHR - Arrival,Jn 1,0.88,38.2,D
1,Two at grade -GHR - Departure,Jn 1,0.97,50.5,D
2,Two at grade - GHR - Arrival,Jn 8,1.01,64.7,E
3,Two at grade - GHR - Departure,Jn 8,0.95,50.8,D
4,Two at grade - GHR - Arrival,Jn 10,0.77,16.9,B
5,Two at grade - GHR - Departure,Jn 10,0.93,33.6,C
6,Two at grade - GHR - Arrival,Jn 11,0.80,6.8,A
7,Two at grade - GHR - Departure,Jn 11,0.87,23.2,C
8,Two at grade- GHR - Arrival,Jn 12,0.60,18.0,B
9,Two at grade- GHR - Departure,Jn 12,0.56,16.0,B


In [ ]:
Output.to_excel('Synchro_Report.xlsx', index=False)